In [1]:
import random
import os
import numpy as np
import torch
import json
from tqdm import tqdm
import pandas as pd
from transformers import RobertaTokenizer
import time
from transformers import AutoTokenizer, AutoModel

In [2]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t")

#merge title into content 
#TODO: we need to first convert any na's to "", that way 


/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,6,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
len(df) - len(df.dropna(subset=["content"]))

33544

In [4]:
#convert NA's to empty strings 
#df[["title", "content"]] = df[["title", "content"]].fillna("").
df["content"] = df["title"] + df["content"] 


In [5]:
#this shows that removing content after merging with title and 
#simply removing all rows with no content will do the same thing
len(df) - len(df.dropna(subset=["content"]))

33544

In [ ]:
#NOTE: very important step, we are removing rows here
df.dropna(subset=["content"])

In [5]:
df["source"].unique()

array(['andalusiastarnews', 'atmoreadvance', 'thebrewtonstandard',
       'clantonadvertiser', 'gadsdenmessenger', 'greenvilleadvocate',
       'selmatimesjournal', 'troymessenger', 'thevalleytimesnews',
       'alaskajournalofcommerce', 'juneauempire', 'peninsulaclarion',
       'arizonacapitoltimes', 'arkansastimes', 'fayettevillefreeweekly',
       'alamedatimesstar', 'thebakersfieldcalifornian',
       'campbellreporter', 'contracostatimes', 'cupertinocourier'],
      dtype=object)

In [ ]:
#we only care about articles from certain sources 

In [60]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [61]:
# truncate text from either head or tail part
def trunc_text(text, trunc_pos, length):

    text_ids = tokenizer.encode(text)[1:-1]

    if trunc_pos == 'head':
        text_trunc_ids = text_ids[:length]
    elif trunc_pos == 'tail':
        text_trunc_ids = text_ids[-length:]

    text_trunc_tokens = tokenizer.decode(text_trunc_ids)
    #text_trunc_back_sent = ''.join([x.replace('_', ' ') for x in text_trunc_tokens])[:-1]

    return text_trunc_tokens

In [62]:
#quick sanity check 
trunc_text(df["content"][0], "tail", 20)

Token indices sequence length is longer than the specified maximum sequence length for this model (1840 > 512). Running this sequence through the model will result in indexing errors


'##vington historical society. she can be reached at suebwilson47 @ gmail. com.'

In [63]:
HEAD_COUNT = 288
TAIL_COUNT = 96

In [66]:
len(df)

4736325

In [64]:
#6 seconds for 1,000 
#1 minute for 10,000
print("starting tokenization")
st = time.time()
df["contentHead"] = df["content"] .apply(trunc_text, args=["head", HEAD_COUNT])
df["contentTail"] = df["content"].apply(trunc_text, args=["tail", TAIL_COUNT])
et = time.time()
print(et - st)

KeyboardInterrupt: 

In [47]:
df["headTail"] = df["contentHead"] + df["contentTail"]

In [48]:
#NOTE: only uncomment if ready to write! always make sure we write to /shared 
#and not /home 
df.to_csv("/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/headTailMerged.tsv", sep="\t")